# Modelagem e previsão

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

from itertools import product
import datetime

## 1. Importando dados 

In [2]:
df = pd.read_csv("../data/dados_unificados.csv", decimal=',')

In [3]:
# Garante que 'Date' esteja no formato datetime
df['Date'] = pd.to_datetime(df['Date'])

# Cria colunas de tempo (se ainda não existirem)
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.isocalendar().week

# Define alvo
target = 'Weekly_Sales'

# Features selecionadas
features = ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
            'Type', 'Size', 'Month', 'Week', 'IsHoliday', 'TemDesconto']


In [4]:

# Codifica 'Type' (categórica)
df['Type'] = LabelEncoder().fit_transform(df['Type'])

# Separa dados de treino (anos anteriores) e teste (ano seguinte)
df_train = df[df['Year'] < df['Year'].max()]
df_test = df[df['Year'] == df['Year'].max()]  # Ano seguinte

X_train = df_train[features]
y_train = df_train[target]
X_test = df_test[features]


NameError: name 'LabelEncoder' is not defined

In [ ]:

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Previsões
df_test['Weekly_Sales_Predicted'] = model.predict(X_test)


In [ ]:
previsoes = df_test.groupby(['Store', 'Dept'])['Weekly_Sales_Predicted'].sum().reset_index()
previsoes.columns = ['Store', 'Dept', 'Predicted_Annual_Sales']


In [ ]:
# Top 10 maiores previsões de vendas
print(previsoes.sort_values(by='Predicted_Annual_Sales', ascending=False).head(10))

# Exportar
# previsoes.to_csv("previsoes_vendas_loja_depto.csv", index=False)


In [5]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
df.sort_values(by='Date', inplace=True)

In [7]:
df.Date.min()

Timestamp('2010-02-05 00:00:00')

In [8]:
df.Date.max()

Timestamp('2012-10-26 00:00:00')

In [9]:
df.columns

Index(['Unnamed: 0', 'Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday',
       'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'Year',
       'Month', 'Week', 'DayOfYear', 'DescontoSum', 'TemDesconto',
       'DescontoAvg'],
      dtype='object')

In [10]:
df[df.IsHoliday == True][["Date",'Week']].drop_duplicates()

,Date,Week
219108,2010-02-12,6
58133,2010-09-10,36
134006,2010-11-26,47
408995,2010-12-31,52
352335,2011-02-11,6
220313,2011-09-09,36
189431,2011-11-25,47
59242,2011-12-30,52
333009,2012-02-10,6
403894,2012-09-07,36


## 2. Treinando modelo de previsão

### 2.1 Separar dados de treino e teste


In [11]:
# Separar dados de treino (tudo exceto o último ano) e teste (ano seguinte)
train_df = df[df['Year'] < df['Year'].max()]
test_df = df[df['Year'] == df['Year'].max()]

columns = ['Store', 'Dept', 'IsHoliday', 'Type', 'Size', 'Month']

X_train = train_df[columns]
y_train = train_df['Weekly_Sales']
X_test = test_df[columns]
y_test = test_df['Weekly_Sales']

### 2.2 Experimentar modelos e features

Nesta etapa serão experimentados 5 tipos de técnicas diferentes, além de que para cada técnicas, serão selecionadas k melhores features para treinamento e teste.
Para cada treinamento, será executada a validação cruzada a fim de avaliar o modelo treinado e, no final da execução, obter o melhor modelo que melhor generalize esses dados e que possa ser utilizado para previsão dos próximos anos.

In [12]:
models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(random_state=42, n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, n_jobs=-1)
}

# 3. Avaliação via cross-validation
results = {}

for name, model in models.items():
    print("="*80)
    print("Model:", name)
    print("="*80)
    
    for k_best in range(4, len(X_train.columns) + 1):
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('select', SelectKBest(score_func=f_regression, k=k_best)),
            ('regressor', model)
        ])
        
        # TimeSeriesSplit para evitar leakage temporal
        tscv = TimeSeriesSplit(n_splits=5)
        
        scores = cross_val_score(pipeline, X_train, y_train, scoring='r2', cv=tscv)
        results[name + "_" + str(k_best)] = {
            'R2_mean': np.mean(scores),
            'R2_std': np.std(scores),
            'K_best': k_best
        }
        
        print("K_best:", str(k_best))
        print("R2:", results[name + "_" + str(k_best)]["R2_mean"])
        print("-"*80)
    print("")

Model: LinearRegression
K_best: 4
R2: 0.08211125464483085
--------------------------------------------------------------------------------
K_best: 5
R2: 0.0830414347133845
--------------------------------------------------------------------------------
K_best: 6
R2: 0.08311002199813822
--------------------------------------------------------------------------------

Model: RandomForest
K_best: 4
R2: 0.8732853235293281
--------------------------------------------------------------------------------
K_best: 5
R2: 0.8957788175505274
--------------------------------------------------------------------------------
K_best: 6
R2: 0.9081493807705382
--------------------------------------------------------------------------------

Model: GradientBoosting
K_best: 4
R2: 0.7170703736126258
--------------------------------------------------------------------------------
K_best: 5
R2: 0.7137768443436773
--------------------------------------------------------------------------------
K_best: 6
R2: 0.

In [13]:
# 4. Treinar o melhor modelo e prever o ano seguinte
best_model_name = max(results, key=lambda x: results[x]['R2_mean'])
best_model = models[best_model_name.split("_")[0]]
k_best = best_model_name.split("_")[1]

final_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('select', SelectKBest(score_func=f_regression, k=int(k_best))),
    ('regressor', best_model)
])

final_pipeline.fit(X_train, y_train)
y_pred = final_pipeline.predict(X_test)

# 5. Avaliação no conjunto do ano seguinte
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

selected_mask = final_pipeline.named_steps['select'].get_support()
selected_columns = X_train.columns[selected_mask]

print("="*80)
print(f"Melhor modelo: {best_model_name}")
print(f"K_Best: {k_best}")
print(f"Colunas: {list(selected_columns)}")
print(f"R² (ano seguinte): {r2:.4f}")
print(f"RMSE (ano seguinte): {rmse:.2f}")

Melhor modelo: DecisionTree_6
K_Best: 6
Colunas: ['Store', 'Dept', 'IsHoliday', 'Type', 'Size', 'Month']
R² (ano seguinte): 0.9665
RMSE (ano seguinte): 4049.87


## 3. Prever 2013

### 3.1 Criar base para previsão

#### 3.1.1 Selecionar dados de lojas e juntar com weeks

In [14]:
# df tem as colunas Store, Dept, Type, Size
df_data = df[["Store", "Dept", "Type", "Size"]].drop_duplicates().copy()
weeks_2013 = pd.DataFrame({'Date': pd.date_range('2013-01-04', periods=52, freq='W-FRI')})

# Fazendo merge de dados das lojas com weeks
future = df_data.merge(weeks_2013, how='cross')
future

,Store,Dept,Type,Size,Date
0,1,1,0,151315,2013-01-04
1,1,1,0,151315,2013-01-11
2,1,1,0,151315,2013-01-18
3,1,1,0,151315,2013-01-25
4,1,1,0,151315,2013-02-01
...,...,...,...,...,...
173207,17,99,1,93188,2013-11-29
173208,17,99,1,93188,2013-12-06
173209,17,99,1,93188,2013-12-13
173210,17,99,1,93188,2013-12-20


#### 3.1.2 Adicionar campo IsHoliday e Month

In [15]:
df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

In [16]:
!pip install holidays


In [17]:
import holidays

# Obter o calendário de feriados dos EUA para 2013
us_holidays = holidays.US(years=2013)

# Função para saber se há feriado na semana (segunda a domingo)
def is_week_holiday(date):
    week_start = date - pd.Timedelta(days=date.weekday())  # segunda
    week_end = week_start + pd.Timedelta(days=6)           # domingo
    return any(week_start <= pd.to_datetime(d) <= week_end for d in us_holidays)

# Aplicar a função
future['IsHoliday'] = future['Date'].apply(is_week_holiday).astype(int)

In [18]:
future

,Store,Dept,Type,Size,Date,IsHoliday
0,1,1,0,151315,2013-01-04,1
1,1,1,0,151315,2013-01-11,0
2,1,1,0,151315,2013-01-18,0
3,1,1,0,151315,2013-01-25,1
4,1,1,0,151315,2013-02-01,0
...,...,...,...,...,...,...
173207,17,99,1,93188,2013-11-29,1
173208,17,99,1,93188,2013-12-06,0
173209,17,99,1,93188,2013-12-13,0
173210,17,99,1,93188,2013-12-20,0


In [19]:
df[["Store", "Dept", "Type", "Size"]].drop_duplicates()

,Store,Dept,Type,Size
0,1,1,0,151315
277665,29,5,1,93638
277808,29,6,1,93638
277951,29,7,1,93638
278094,29,8,1,93638
...,...,...,...,...
343680,36,49,0,39910
246621,25,96,1,128107
117605,12,99,1,112238
393620,42,24,2,39690


In [20]:
# Features temporais
future['Year'] = future['Date'].dt.year
future['Month'] = future['Date'].dt.month
future['Week'] = future['Date'].dt.isocalendar().week.astype(int)

In [21]:
future

,Store,Dept,Type,Size,Date,IsHoliday,Year,Month,Week
0,1,1,0,151315,2013-01-04,1,2013,1,1
1,1,1,0,151315,2013-01-11,0,2013,1,2
2,1,1,0,151315,2013-01-18,0,2013,1,3
3,1,1,0,151315,2013-01-25,1,2013,1,4
4,1,1,0,151315,2013-02-01,0,2013,2,5
...,...,...,...,...,...,...,...,...,...
173207,17,99,1,93188,2013-11-29,1,2013,11,48
173208,17,99,1,93188,2013-12-06,0,2013,12,49
173209,17,99,1,93188,2013-12-13,0,2013,12,50
173210,17,99,1,93188,2013-12-20,0,2013,12,51


In [22]:
features = ['Store', 'Dept', 'IsHoliday', 'Type', 'Size', 'Month']

In [23]:
future[features]

,Store,Dept,IsHoliday,Type,Size,Month
0,1,1,1,0,151315,1
1,1,1,0,0,151315,1
2,1,1,0,0,151315,1
3,1,1,1,0,151315,1
4,1,1,0,0,151315,2
...,...,...,...,...,...,...
173207,17,99,1,1,93188,11
173208,17,99,0,1,93188,12
173209,17,99,0,1,93188,12
173210,17,99,0,1,93188,12


### 3.2 Previsão

In [24]:
future['Predicted_Sales'] = final_pipeline.predict(future[features])

In [25]:
future

,Store,Dept,Type,Size,Date,IsHoliday,Year,Month,Week,Predicted_Sales
0,1,1,0,151315,2013-01-04,1,2013,1,1,41963.330000
1,1,1,0,151315,2013-01-11,0,2013,1,2,17286.647500
2,1,1,0,151315,2013-01-18,0,2013,1,3,17286.647500
3,1,1,0,151315,2013-01-25,1,2013,1,4,41963.330000
4,1,1,0,151315,2013-02-01,0,2013,2,5,28966.508333
...,...,...,...,...,...,...,...,...,...,...
173207,17,99,1,93188,2013-11-29,1,2013,11,48,869.565000
173208,17,99,1,93188,2013-12-06,0,2013,12,49,266.787500
173209,17,99,1,93188,2013-12-13,0,2013,12,50,266.787500
173210,17,99,1,93188,2013-12-20,0,2013,12,51,266.787500


## 4. Salvar modelo

### 4.1 Salvar melhor modelo

In [29]:
import joblib

joblib.dump(final_pipeline, '../model/model.pkl')

['../model/model.pkl']

### 3.2 Criar mapas de apoio ao modelo

#### 3.2.1 Mapa com features da loja 

In [59]:
df[['Store', 'Dept', 'Type', 'Size']][df.Store == 1].drop_duplicates()

,Store,Dept,Type,Size
0,1,1,0,151315
6194,1,49,0,151315
6069,1,48,0,151315
6047,1,47,0,151315
6337,1,51,0,151315
...,...,...,...,...
2687,1,21,0,151315
7936,1,78,0,151315
9799,1,96,0,151315
10207,1,99,0,151315


In [61]:
store_dict = df[['Store', 'Type', 'Size']].drop_duplicates().set_index('Store')[['Type', 'Size']].to_dict(orient='index')
store_dict

{1: {'Type': 0, 'Size': 151315},
 29: {'Type': 1, 'Size': 93638},
 2: {'Type': 0, 'Size': 202307},
 28: {'Type': 0, 'Size': 206302},
 27: {'Type': 0, 'Size': 204184},
 31: {'Type': 0, 'Size': 203750},
 32: {'Type': 0, 'Size': 203007},
 30: {'Type': 2, 'Size': 42988},
 24: {'Type': 0, 'Size': 203819},
 3: {'Type': 1, 'Size': 37392},
 25: {'Type': 1, 'Size': 128107},
 23: {'Type': 1, 'Size': 114533},
 26: {'Type': 0, 'Size': 152513},
 39: {'Type': 0, 'Size': 184109},
 40: {'Type': 0, 'Size': 155083},
 38: {'Type': 2, 'Size': 39690},
 37: {'Type': 2, 'Size': 39910},
 41: {'Type': 0, 'Size': 196321},
 42: {'Type': 2, 'Size': 39690},
 34: {'Type': 0, 'Size': 158114},
 33: {'Type': 0, 'Size': 39690},
 36: {'Type': 0, 'Size': 39910},
 35: {'Type': 1, 'Size': 103681},
 11: {'Type': 0, 'Size': 207499},
 4: {'Type': 0, 'Size': 205863},
 12: {'Type': 1, 'Size': 112238},
 10: {'Type': 1, 'Size': 126512},
 13: {'Type': 0, 'Size': 219622},
 14: {'Type': 0, 'Size': 200898},
 6: {'Type': 0, 'Size': 20

In [72]:
import pickle

with open('../model/stores.pkl', 'wb') as f:
    pickle.dump(store_dict, f)

In [57]:
store = 1

In [58]:
store_dict[store]

{'Type': 0, 'Size': 151315}

#### 3.2.2 Mapa com Depts

In [62]:
df[['Store', 'Dept']].drop_duplicates()

,Store,Dept
0,1,1
277665,29,5
277808,29,6
277951,29,7
278094,29,8
...,...,...
343680,36,49
246621,25,96
117605,12,99
393620,42,24


In [69]:
store_dpto_dict = df.groupby('Store')['Dept'].unique().to_dict()
store_dpto_dict

{1: array([ 1, 49, 48, 47, 51, 54, 52, 42, 41, 44, 46, 45, 55, 71, 67, 74, 72,
        58, 56, 60, 59, 28, 27, 30, 29, 26, 23, 22, 24, 25, 37, 36, 40, 38,
        35, 32, 31, 33, 34, 79, 87, 85, 91, 90, 81, 80, 92, 83, 82, 98, 97,
        94, 93, 95, 19,  6, 16, 17,  5, 13,  7, 14,  9, 11, 20,  4, 18,  2,
         3, 12,  8, 10, 21, 78, 96, 99, 77]),
 2: array([49, 48, 54, 52, 51, 42, 41, 46, 45, 44, 55, 71, 67, 72, 74, 58, 56,
        59, 78, 28, 27, 30, 29, 24, 23, 26, 25, 37, 36, 40, 38, 32, 31, 35,
        33, 34, 79, 87, 85, 91, 90, 81, 80, 83, 82, 92, 98, 97, 94, 93, 96,
        95, 22,  9,  8, 11, 10,  4,  5,  7,  6, 19, 18, 21, 20, 13, 12, 17,
        14, 16,  3,  2,  1, 39, 60, 47, 99, 77]),
 3: array([ 9,  8, 11, 10,  4,  5,  7,  6, 12, 19, 18, 21, 20, 14, 13, 17, 16,
         3,  2,  1, 22, 55, 54, 56, 60, 59, 46, 45, 44, 52, 51, 47, 81, 82,
        87, 79, 85, 71, 67, 74, 72, 28, 27, 30, 29, 31, 23, 24, 26, 25, 38,
        40, 42, 36, 41, 33, 32, 35, 34, 97, 96, 91, 90, 95,

In [73]:
import pickle

with open('../model/depts.pkl', 'wb') as f:
    pickle.dump(store_depts, f)